# 🪄 Quantization Tutorial

Quantization is a technique to reduce the computational and memory costs of running inference by representing the weights and activations with low-precision data types like 8-bit integer (`int8`) instead of the usual 32-bit floating point (`float32`).

Reducing the number of bits means the resulting model requires less memory storage, consumes less energy (in theory), and operations like matrix multiplication can be performed much faster with integer arithmetic. It also allows to run models on embedded devices, which sometimes only support integer data types.

In [ ]:
!pip install tensorflow-model-optimization

Imports

In [2]:
import os
import shutil
import random
import numpy as np
from zipfile import ZipFile as unzip
from google.colab import files
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array, load_img

Dataset

In this Tutorial we will be building a Binary Image Classification Model for Cats vs Dogs.

The dataset is taken from kaggle for ease of use.

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"srddev","key":"7eb7452f953af3d3dafc92a688f98ed9"}'}

In [5]:
! pip install kaggle
! mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c dogs-vs-cats

 99% 807M/812M [00:05<00:00, 248MB/s]
100% 812M/812M [00:05<00:00, 157MB/s]


In [7]:
for _ , _  , filenames in os.walk('.'):
  for filename in filenames:
    if filename[-3:]=='zip':
      file = unzip(filename,'r')
      file.extractall()
      file.close()
      os.remove(filename)

os.remove('sampleSubmission.csv')

In [8]:
for _ , _  , filenames in os.walk('.'):
  for filename in filenames:
    if filename[-3:]=='zip':
      file = unzip(filename,'r')
      file.extractall()
      file.close()
      os.remove(filename)

In [9]:
print("Number of train images: " , len(os.listdir('./train')) )
print("Number of test images: "  , len(os.listdir('./test1')) )

Number of train images:  25000
Number of test images:  12500


In [10]:
no_of_cats = len([files for files in os.listdir('./train') if 'cat' in files])
no_of_dogs = len([files for files in os.listdir('./train') if 'dog' in files])
print("Cat images in train dataset: ",  no_of_cats)
print("Dog images in train dataset: ", no_of_dogs)
if no_of_cats==no_of_dogs:
  total_images = no_of_cats
  print('Balanced dataset')

Cat images in train dataset:  12500
Dog images in train dataset:  12500
Balanced dataset


Create a folder named `data` which has 3 sub folders namely
- train
- validation
- test

In [11]:
base_dir = 'data/'
dirs = ['train','val','test']
sub_dirs = ['cats','dogs']
for dir in dirs:
  dir =  os.path.join(base_dir, dir)
  os.makedirs(dir,exist_ok=True)
  for sub_dir in sub_dirs:
    sub_dir = os.path.join(dir, sub_dir)
    os.makedirs(sub_dir,exist_ok=True)

In [12]:
split = {'train': 0.7,
         'val': 0.3}

for dir in dirs[:2]:
    print(int(split[dir]*total_images))

8750
3750


In [13]:
split_size = 0.7

split = {'train': (0,int(split_size*total_images)),
         'val': (int(split_size*total_images),total_images)}

for obj in sub_dirs:
  for dir in dirs[:2]:
    files = ['{}.{}.jpg'.format(obj[:-1],i) for i in range(*split[dir])]
    for filename in files:
      src = os.path.join('./train', filename)
      dst = os.path.join(base_dir,dir,obj,filename)
      shutil.copyfile(src,dst)
print("Train validation split completed.")

Train validation split completed.


In [14]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [15]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

['cat.258.jpg', 'cat.2649.jpg', 'cat.7962.jpg', 'cat.6546.jpg', 'cat.6809.jpg', 'cat.7419.jpg', 'cat.5057.jpg', 'cat.1010.jpg', 'cat.7380.jpg', 'cat.2358.jpg']
['dog.6198.jpg', 'dog.2620.jpg', 'dog.6557.jpg', 'dog.8016.jpg', 'dog.8122.jpg', 'dog.7579.jpg', 'dog.6551.jpg', 'dog.8035.jpg', 'dog.4384.jpg', 'dog.2017.jpg']


In [16]:
print('total training cat images :', len(os.listdir(train_cats_dir ) ))
print('total training dog images :', len(os.listdir(train_dogs_dir ) ))

print('total validation cat images :', len(os.listdir(validation_cats_dir )))
print('total validation dog images :', len(os.listdir(validation_dogs_dir )))

total training cat images : 8750
total training dog images : 8750
total validation cat images : 3750
total validation dog images : 3750


Model Building

In [17]:
model = tf.keras.Sequential([
        Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)),
        Conv2D(64,kernel_size=(3,3),activation='relu'),
        MaxPooling2D(4,4),

        Conv2D(128,kernel_size=(3,3),activation='relu'),
        Conv2D(64,kernel_size=(3,3),activation='relu'),
        MaxPooling2D(4,4),

        Flatten(),
        Dense(512,activation='relu') ,
        Dropout(rate=0.3),
        Dense(1, activation='sigmoid'),
        ])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 128)     3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 64)      73792     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 36, 36, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                    

In [19]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [20]:
datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = datagen.flow_from_directory(train_dir,
                                              batch_size=20,
                                              class_mode ='binary',
                                              target_size=(150,150))
validation_generator = datagen.flow_from_directory(validation_dir,
                                                   batch_size=20,
                                                   class_mode='binary',
                                                   target_size = (150,150))

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.


In [ ]:
histroy = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch=100,
                    epochs=50,
                    validation_steps = 50,
                    verbose=1)

Epoch 1/50
100/100 [==============================] - 21s 79ms/step - loss: 0.7167 - accuracy: 0.5070 - val_loss: 0.6910 - val_accuracy: 0.5100
Epoch 2/50
100/100 [==============================] - 8s 83ms/step - loss: 0.6955 - accuracy: 0.5410 - val_loss: 0.6904 - val_accuracy: 0.5030
Epoch 3/50
100/100 [==============================] - 7s 73ms/step - loss: 0.6852 - accuracy: 0.5520 - val_loss: 0.6708 - val_accuracy: 0.6160
Epoch 4/50
100/100 [==============================] - 11s 109ms/step - loss: 0.6563 - accuracy: 0.6180 - val_loss: 0.6223 - val_accuracy: 0.6360
Epoch 5/50
100/100 [==============================] - 9s 85ms/step - loss: 0.6319 - accuracy: 0.6380 - val_loss: 0.6217 - val_accuracy: 0.6370
Epoch 6/50
100/100 [==============================] - 12s 120ms/step - loss: 0.6184 - accuracy: 0.6660 - val_loss: 0.5914 - val_accuracy: 0.6860
Epoch 7/50
100/100 [==============================] - 12s 119ms/step - loss: 0.5698 - accuracy: 0.7080 - val_loss: 0.5990 - val_accuracy:

save the trained model in a directory `saved_models`

In [24]:
model.save("saved_models/")

In [25]:
import math

def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

## 2. Post Training Quantization

Coverting the saved model into a tflite model without **Quantization**

In [26]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_models/")
tflite_model = converter.convert()
print(convert_size(len(tflite_model)))

8.87 MB


Converting the saved model into a tflite model with **Quantization**

In [27]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_models/")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
print(convert_size(len(tflite_quant_model)))

2.23 MB


In [28]:
with open("tflite_model.tflite", "wb") as f:
    f.write(tflite_model)

with open("tflite_quant_model.tflite", "wb") as f:
    f.write(tflite_quant_model)

## 3. Quantization aware training

Fine-tuning the trained model using Quantization aware training and converting it into tflite model.

In [29]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer=RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics = ['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 150, 150, 3)      3         
 er)                                                             
                                                                 
 quant_conv2d (QuantizeWrapp  (None, 148, 148, 128)    3843      
 erV2)                                                           
                                                                 
 quant_conv2d_1 (QuantizeWra  (None, 146, 146, 64)     73923     
 pperV2)                                                         
                                                                 
 quant_max_pooling2d (Quanti  (None, 36, 36, 64)       1         
 zeWrapperV2)                                                    
                                                                 
 quant_conv2d_2 (QuantizeWra  (None, 34, 34, 128)      7

In [30]:
q_aware_model.fit(train_generator,validation_data = validation_generator,steps_per_epoch=100,epochs=5,validation_steps = 10,verbose=1)

Epoch 1/5
100/100 [==============================] - 15s 118ms/step - loss: 0.2164 - accuracy: 0.9175 - val_loss: 0.3407 - val_accuracy: 0.8700
Epoch 2/5
100/100 [==============================] - 11s 111ms/step - loss: 0.2259 - accuracy: 0.9160 - val_loss: 0.4573 - val_accuracy: 0.8450
Epoch 3/5
100/100 [==============================] - 10s 100ms/step - loss: 0.2283 - accuracy: 0.9105 - val_loss: 0.4861 - val_accuracy: 0.8100
Epoch 4/5
100/100 [==============================] - 10s 100ms/step - loss: 0.1892 - accuracy: 0.9355 - val_loss: 0.5461 - val_accuracy: 0.8450
Epoch 5/5
100/100 [==============================] - 10s 98ms/step - loss: 0.2203 - accuracy: 0.9165 - val_loss: 0.4263 - val_accuracy: 0.8300


In [31]:
test_dir = os.path.join(base_dir, 'test')
test_generator = datagen.flow_from_directory(test_dir,
                                                   batch_size=10,
                                                   class_mode='binary',
                                                   target_size = (150,150))

Found 0 images belonging to 2 classes.


In [33]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_qaware_model = converter.convert()

len(tflite_qaware_model)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


2341456

In [34]:
print(convert_size(len(tflite_qaware_model)))

2.23 MB


In [35]:
with open("tflite_qaware_model.tflite", 'wb') as f:
    f.write(tflite_qaware_model)